In [9]:
import os
import json
from io import BytesIO
from dotenv import load_dotenv

import requests
import pandas as pd
from glob import glob
from tqdm import tqdm
from PIL import Image
from minio import Minio
from minio.error import S3Error

In [10]:
load_dotenv()

True

In [11]:
client = Minio(
    os.environ.get('S3_ENDPOINTS'),
    access_key=os.environ.get('S3_ACCESS_KEY'),
    secret_key=os.environ.get('S3_SECRET_KEY'),
    secure=False
)

In [12]:
parsed_list = glob("./data/semi-structured/*.json")
parsed_list

['./data/semi-structured/홍대 회식 맛집.json',
 './data/semi-structured/홍대 데이트 맛집.json',
 './data/semi-structured/성수 데이트 맛집.json',
 './data/semi-structured/압구정 회식 맛집.json',
 './data/semi-structured/강남역 데이트 맛집.json',
 './data/semi-structured/영등포 회식 맛집.json',
 './data/semi-structured/성수 회식 맛집.json',
 './data/semi-structured/압구정 데이트 맛집.json',
 './data/semi-structured/강남역 회식 맛집.json',
 './data/semi-structured/범계 회식 맛집.json',
 './data/semi-structured/범계 데이트 맛집.json',
 './data/semi-structured/영등포 데이트 맛집.json']

In [21]:
def load_img(url: str):
    res = requests.get(url)
    img = Image.open(BytesIO(res.content))
    
    return img

In [23]:
img = load_img('https://postfiles.pstatic.net/MjAyMzAyMjVfOCAg/MDAxNjc3MzE3ODY2MjYz.xVfavTXtn1K6evITcB1lbTdljYyr34S0bTAG7K3H0W8g.5SpztDyBDQJ_PVupIW_UAXfKR7luiVE6Nmhc-e3orA4g.JPEG.shuenmama/IMG_2027.jpg?type=w773')

In [25]:
bucket_name = 'jcwee.study'
file_path = 'skku/capstone/fake_review_detect/data/photo/{article_id}/{no}.png'

img_byte = BytesIO()
img.save(img_byte, format='PNG')
img_byte = img_byte.getvalue()

client.put_object(
    bucket_name,
    file_path,
    data=BytesIO(img_byte),
    length=len(img_byte),
    content_type='image/png'
)

In [27]:
bucket_name = 'jcwee.study'
file_path = 'skku/capstone/fake_review_detect/data/photo/{article_id}/{no}.png'

for path in parsed_list:
    with open(path, 'r', encoding='utf-8') as f:
        blog = json.load(f)
    
    for article in tqdm(blog):
        article_id = '.'.join(article['url'].split('/')[-2:])
        
        for p in article['contents']:
            if p['info'] in ['img', 'gif']:
                try:
                    img = load_img(p['content'])

                    img_byte = BytesIO()
                    img.save(img_byte, format='PNG')
                    img_byte = img_byte.getvalue()

                    client.put_object(
                        bucket_name,
                        file_path.format(article_id=article_id, no=p['no']),
                        data=BytesIO(img_byte),
                        length=len(img_byte),
                        content_type='image/png'
                    )
                    
                except Exception as e:
                    print(e)

  0%|          | 2/888 [00:31<3:54:22, 15.87s/it]


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7f877c334090>

In [28]:
p

{'no': 3,
 'info': 'img',
 'content': 'https://postfiles.pstatic.net/MjAyMzA5MTJfNzMg/MDAxNjk0NDg3NDA4MzM0.qtK7_b3-T_QE-c-KNLQ4WaxRSCRu-ni7auQndPTN98kg.n7LjtbG74HJCsXDIGS7JzNMPM8oylUl3o9YxeCC3RDYg.JPEG.mou25/1.jpg?type=w773'}

In [29]:
load_img('https://postfiles.pstatic.net/MjAyMzA5MTJfNzMg/MDAxNjk0NDg3NDA4MzM0.qtK7_b3-T_QE-c-KNLQ4WaxRSCRu-ni7auQndPTN98kg.n7LjtbG74HJCsXDIGS7JzNMPM8oylUl3o9YxeCC3RDYg.JPEG.mou25/1.jpg?type=w773')

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7f877a65ef70>